<a href="https://colab.research.google.com/github/gsulav2041/Sulav_DTSC3020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [2]:
 #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [3]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [7]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [8]:
# Q1 — Write your answer here


import pandas as pd

# --- Q1 Implementation ---

def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML."""
    # Read all tables from the page
    tables = pd.read_html(html)

    # Find the first table with at least 3 columns
    for t in tables:
        if t.shape[1] >= 3:
            df = t.copy()
            break
    else:
        raise ValueError("No table found with >= 3 columns.")

    # Flatten headers (if multi-level)
    df.columns = [str(c).strip() for c in df.columns]
    return df


def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids."""

    # Standardize expected column names
    rename_map = {
        'Alpha-2 code': 'Alpha-2',
        'Alpha-3 code': 'Alpha-3',
        'Numeric code': 'Numeric'
    }
    df = df.rename(columns=rename_map)

    # Keep only relevant columns (add others if desired)
    cols = ['Country', 'Alpha-2', 'Alpha-3', 'Numeric']
    df = df[cols].copy()

    # Clean string fields
    df['Country'] = df['Country'].astype(str).str.strip()
    df['Alpha-2'] = df['Alpha-2'].astype(str).str.strip().str.upper()
    df['Alpha-3'] = df['Alpha-3'].astype(str).str.strip().str.upper()

    # Convert Numeric to nullable int
    df['Numeric'] = pd.to_numeric(df['Numeric'], errors='coerce').astype('Int64')

    # Drop invalid rows (e.g., missing country or numeric code)
    df = df.dropna(subset=['Country', 'Alpha-2', 'Alpha-3', 'Numeric'])

    return df


def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N."""
    df_sorted = df.sort_values(by='Numeric', ascending=False).head(top)
    return df_sorted


# --- Example Run ---
if __name__ == "__main__":
    url = "https://www.iban.com/country-codes"
    df_raw = q1_read_table(url)
    df_clean = q1_clean(df_raw)
    df_top = q1_sort_top(df_clean)

    # Write to CSV
    df_clean.to_csv("data_q1.csv", index=False)
    print(df_top)




                                               Country Alpha-2 Alpha-3  \
247                                             Zambia      ZM     ZMB   
246                                              Yemen      YE     YEM   
192                                              Samoa      WS     WSM   
244                                  Wallis and Futuna      WF     WLF   
240                 Venezuela (Bolivarian Republic of)      VE     VEN   
238                                         Uzbekistan      UZ     UZB   
237                                            Uruguay      UY     URY   
35                                        Burkina Faso      BF     BFA   
243                              Virgin Islands (U.S.)      VI     VIR   
236                     United States of America (the)      US     USA   
219                       Tanzania, United Republic of      TZ     TZA   
108                                        Isle of Man      IM     IMN   
113                                   

## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [10]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [11]:
# Q2 — Write your answer here
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# --- Q2 Implementation ---

def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    """
    soup = BeautifulSoup(html, "html.parser")
    rows = soup.select("tr.athing")
    data = []

    for row in rows:
        # Rank, title, link
        rank_tag = row.select_one(".rank")
        title_tag = row.select_one(".titleline a")

        rank = rank_tag.text.replace(".", "").strip() if rank_tag else "0"
        title = title_tag.text.strip() if title_tag else ""
        link = title_tag["href"] if title_tag else ""

        # Subtext row is the next sibling <tr>
        sub = row.find_next_sibling("tr").select_one(".subtext")
        if sub:
            # Points, user, comments
            points_tag = sub.select_one(".score")
            user_tag = sub.select_one(".hnuser")
            comment_tag = sub.find_all("a")[-1]

            points = points_tag.text if points_tag else "0 points"
            user = user_tag.text if user_tag else ""
            comments_text = comment_tag.text if comment_tag else "0 comments"
        else:
            points, user, comments_text = "0 points", "", "0 comments"

        data.append({
            "rank": rank,
            "title": title,
            "link": link,
            "points": points,
            "comments": comments_text,
            "user": user
        })

    return pd.DataFrame(data)


def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values."""
    def to_int(x):
        # Extract digits, default 0 if none
        m = re.search(r"\d+", str(x))
        return int(m.group()) if m else 0

    for col in ["rank", "points", "comments"]:
        df[col] = df[col].apply(to_int)

    # Fill missing text fields
    for col in ["title", "link", "user"]:
        df[col] = df[col].fillna("").astype(str).str.strip()

    return df


def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N."""
    return df.sort_values(by="points", ascending=False).head(top)


# --- Example Run ---
if __name__ == "__main__":
    url = "https://news.ycombinator.com/"
    html = requests.get(url).text

    df_raw = q2_parse_items(html)
    df_clean = q2_clean(df_raw)
    df_top = q2_sort_top(df_clean)

    # Save to CSV
    df_clean.to_csv("data_q2.csv", index=False)

    print(df_top)




    rank                                              title  \
10    11                          You should write an agent   
1      2                           Leaving Meta and PyTorch   
15    16           Two billion email addresses were exposed   
21    22  Show HN: I scraped 3B Goodreads reviews to tra...   
3      4                                    A Fond Farewell   
22    23                              Game design is simple   
13    14  Meta projected 10% of 2024 revenue came from s...   
28    29  Analysis indicates that the universe’s expansi...   
7      8  OpenMW 0.50.0 Released – open-source Morrowind...   
29    30  From web developer to database developer in 10...   
0      1                                       I Love OCaml   
19    20             Text case changes the size of QR codes   
11    12                 We chose OCaml to write Stategraph   
6      7                                     PyTorch Helion   
8      9    A.I. and Social Media Contribute to 'Brain 